In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
cd drive

/content/drive


In [0]:
ls

class.txt
Code/
Colab Notebooks/
database/
Data Sets/
_detection reference/
drive/
Generating Enviornment Map (Reads)/
Google self driving car monthely report/
Initial tests/
Market Products and Raw Ideas/
Model_one.py
naya_folder_kuki naam nai sujh rha/
Our Product Draft/
pascal_data/
REFERENCES/
requirements.txt
Screen Shot 2018-07-02 at 8.38.24 PM.png
testModel.ipynb
tf1.png
Timeline/
Untitled document.odt


In [0]:
cd pascal_data

/content/drive/pascal_data


In [0]:
ls

ePascalVOCClassifier_epoch18above.h5       pascal_model_epoch20_.h5
ePascalVOCClassifier_epoch18aboveV_1_2.h5  pascalX.npy
ePascalVOCClassifier.h5                    pascalX_val.npy
ePascalVOCClassifier_monday_one.h5         pascaly.npy
littleClassifier.h5                        pascaly_val.npy
pascal_model_epoch10.h5


In [0]:
import numpy as np

X = np.load('pascalX.npy')
Y = np.load('pascaly.npy')

print(X.shape)
print(Y.shape)

(16551, 300, 300, 3)
(16551, 20)


In [0]:
X_val = np.load('pascalX_val.npy')
Y_val = np.load('pascaly_val.npy')

In [0]:
# import modules
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers.normalization import BatchNormalization
from keras import optimizers
import keras


Using TensorFlow backend.


In [0]:
x = Sequential()

# Block 1
x.add(Conv2D(64, (1, 1),activation='relu',padding='same',input_shape=(300,300,3)))
x.add(Conv2D(64, (1, 1),activation='relu',padding='same'))
x.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

# Block 2
x.add(Conv2D(64, (2, 2),activation='relu',padding='same'))
x.add(Conv2D(64, (2, 2),activation='relu',padding='same'))
x.add(Conv2D(64, (2, 2),activation='relu',padding='same'))
x.add(Conv2D(64, (2, 2),activation='relu',padding='same'))
x.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))
x.add(BatchNormalization())


# Block 3
x.add(Conv2D(128, (2, 2),activation='relu',padding='same'))
x.add(Conv2D(128, (1, 1),activation='relu',padding='same'))

x.add(MaxPooling2D((2, 2), strides=(2, 2)))
x.add(Dropout(0.2))



# Classification block
x.add(Flatten())

# x.add(Dense(1000, activation='relu'))
x.add(Dense(800, activation='relu'))
x.add(Dense(20, activation='softmax', name='predictions'))




In [0]:
# original parameters (lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

# original parameters (lr=0.02, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
nadam = optimizers.Nadam(lr=0.02, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

# original parameters (lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

# categorical_crossentropy
# mean_squared_error
x.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

In [0]:
for i in range(15):
  print("epoch",i+1)
  x.fit(X, Y,epochs=1,batch_size=5,validation_data=(X_val, Y_val))
  x.save('littleClassifier.h5')
  print("----------------------------------------------------------------------")
  print("----------------------------------------------------------------------")


epoch 1
Train on 16551 samples, validate on 4952 samples
Epoch 1/1
 3240/16551 [====>.........................] - ETA: 14:04 - loss: 18.7398 - acc: 0.1228

16550/16551 [============================>.] - ETA: 0s - loss: 18.6619 - acc: 0.1272

16551/16551 [==============================] - 1089s 66ms/step - loss: 18.6618 - acc: 0.1272 - val_loss: 18.0157 - val_acc: 0.1327
----------------------------------------------------------------------
----------------------------------------------------------------------
epoch 2
Train on 16551 samples, validate on 4952 samples
Epoch 1/1
  970/16551 [>.............................] - ETA: 16:18 - loss: 18.3115 - acc: 0.1485

16550/16551 [============================>.] - ETA: 0s - loss: 18.6619 - acc: 0.1272

16551/16551 [==============================] - 1085s 66ms/step - loss: 18.6618 - acc: 0.1272 - val_loss: 18.0157 - val_acc: 0.1327
----------------------------------------------------------------------
----------------------------------------------------------------------
epoch 3
Train on 16551 samples, validate on 4952 samples
Epoch 1/1
  970/16551 [>.............................] - ETA: 16:18 - loss: 18.9928 - acc: 0.1289

16550/16551 [============================>.] - ETA: 0s - loss: 18.6610 - acc: 0.1272

16551/16551 [==============================] - 1085s 66ms/step - loss: 18.6618 - acc: 0.1272 - val_loss: 18.0157 - val_acc: 0.1327
----------------------------------------------------------------------
----------------------------------------------------------------------
epoch 4
Train on 16551 samples, validate on 4952 samples
Epoch 1/1
  970/16551 [>.............................] - ETA: 16:20 - loss: 18.7269 - acc: 0.1258

16550/16551 [============================>.] - ETA: 0s - loss: 18.6619 - acc: 0.1272

16551/16551 [==============================] - 1086s 66ms/step - loss: 18.6618 - acc: 0.1272 - val_loss: 18.0157 - val_acc: 0.1327
----------------------------------------------------------------------
----------------------------------------------------------------------
epoch 5
Train on 16551 samples, validate on 4952 samples
Epoch 1/1
  970/16551 [>.............................] - ETA: 16:16 - loss: 18.6937 - acc: 0.1361

11025/16551 [==================>...........] - ETA: 5:42 - loss: 18.7335 - acc: 0.1259